In [1]:
#sjs 2019 Dec 2

#function to calculate the area of a flare given stellar properties
#and the flare amplitude
import numpy as np
import pandas as pd

from funcs.model import calculate_relative_flare_area, black_body_spectrum, dot_ensemble, lambert

from altaipony.fakeflares import aflare

import astropy.units as u
from astropy.constants import R_sun

# Read in response curve:
response_curve = {"TESS":"TESS.txt",
                       "Kepler":"kepler_lowres.txt"}
for key, val in response_curve.items():
    df = pd.read_csv(f"static/{val}",
                     delimiter="\s+", skiprows=8)
    df = df.sort_values(by="nm", ascending=True)
    rwav = (df.nm * 10).values * u.angstrom #convert to angstroms
    rres = (df.response).values
    response_curve[key] = (rwav,rres)
    
import matplotlib.pyplot as plt
%matplotlib inline

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
# dist, rad, qflux, amp, mission, flaret=1e4
rel_a = calculate_relative_flare_area(50, .1, 1e-12, 10, "TESS", flaret=1e4)

In [ ]:
def calculate_specific_flare_flux(qlum, mission, flaret=1e4):
    """Get the flare area in rel. unit
    
    Parameters:
    -----------
    qlum : float
        quiescent lum in erg/s/cm^2
    mission : string
        TESS or Kepler
    flaret : float
        flare black body temperature, default 10kK
    """
    # Give units to everything:
    qlum = qlum * u.erg/u.s
    
    # Read in response curve:
    rres, rwav = response_curve[mission]
    
    # create an array to upsample the filter curve to
    w = np.arange(3000,13001) * u.angstrom

    # interpolate thermal spectrum onto response 
    # curve wavelength array, then sum up
    # flux times response curve:

    # Generate a thermal spectrum at the flare 
    # temperature over an array of wavelength w:
    thermf = black_body_spectrum(w, flaret) 
    
    # Interpolate response from rwav to w:
    rres = np.interp(w,rwav,rres)
    
    # Integrating the flux of the thermal 
    # spectrum times the response curve over wavelength:
    return np.trapz(thermf * rres, x=w)
    

In [ ]:
radius = 1.12#degree
rdi = np.linspace(0.01,90,200)
As = []

In [ ]:
qlum = 1e30 * u.erg/ u.s
R = 0.1*R_sun
lat, lon = (np.pi/4, 0)
i = 80 * np.pi / 180
a =12
Fth = calculate_specific_flare_flux(qlum, mission="TESS") 
for radius in rdi:
    lats, lons = dot_ensemble(lat, lon, radius, num_pts=1e6)
    N = len(lats)
    A = (N * a * qlum) / (Fth * np.sum(lambert(lons, i, lats, phi0=0.)))
    rnew = (np.arcsin(np.sqrt( A / (4 * np.pi * R**2) )) * 2.).to("deg").value
   # print(rnew,radius)
    As.append(rnew)
   

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(rdi,As)
#plt.plot(np.array([0.1,10.]),np.array([.1,10.]))
plt.xlim(0,4)
#plt.ylim(0.2,.3)
plt.xlabel("model radius [deg]")
plt.ylabel("science radius [deg]")

In [ ]:
((max(As)-min(As))/np.mean(As))*100

In [ ]:
max(As), min(As)

In [ ]:
#A/np.pi/(0.1*R_sun)**2)
R = 0.1*R_sun
A.decompose()

In [ ]:
(A/4/np.pi/R**2).decompose()

In [ ]:
radius = (np.arcsin(np.sqrt( A / (4 * np.pi * R**2) )) * 2.).to("deg")

In [ ]:
radius